In [1]:
pip install pandas akshare loguru

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   ---------------------------------------- 1.1/1.1 MB 10.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/13.7 MB ? eta -:--:--
   ------ --------------------------------- 2.4/13.7 MB 16.8 MB/s eta 0:00:01
   --------------- ------------------------ 5.2/13.7 MB 13.3 MB/s eta 0:00:01
   ------------------------- -------------- 8.7/13.7 MB 14.5 MB/s eta 0:00:01
   ------------------------------------- -- 12.8/13.7 MB 15.8 MB/s eta 0:00:01
   ---------------------------------------- 13.7/13.7 MB 15.3 MB/s eta 0:00:00
  Created wheel for jsonpath: filename=jsonpath-0.82.2-py3-none-any.whl size=5627 sha256=3843c6f023d6a83bc996d8ab965338d6cb4f2555ff93079d37f58b8db196b119
  Stored in directory: c:\users\admin\appdata\local\pip\cache\wheels\73\76\e2\980a29341fe37a583ada29594ed529708d5e8e2c0f9d97c3cc
Success

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
import os
import time
from datetime import datetime
import json
from pathlib import Path
import random
import logging

# 设置中文显示
plt.rcParams["font.family"] = ["SimHei", "WenQuanYi Micro Hei", "Heiti TC"]
plt.rcParams["axes.unicode_minus"] = False  # 解决负号显示问题

# 配置日志
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("stock_analysis.log"),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

class ChinaStockAnalyzer:
    def __init__(self, data_dir="data", output_dir="output", use_proxy=False):
        """初始化分析器，设定数据存储和输出目录"""
        self.data_dir = self._normalize_path(data_dir)
        self.output_dir = self._normalize_path(output_dir)
        self.sse_url = "http://www.sse.com.cn/market/stockdata/statistic/"
        self.szse_url = "http://www.szse.cn/market/overview/index.html"
        self.bse_url = "https://www.bse.cn/static/statisticdata.html"
        self.bse_stat_url = "https://www.bse.cn/static/statisticdata.html"
        self.use_proxy = use_proxy
        
        # 常用请求头，模拟浏览器行为
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
            'Accept-Language': 'zh-CN,zh;q=0.9,en;q=0.8',
            'Connection': 'keep-alive',
        }
        
        # 可选代理池（需要自己准备可用的代理）
        self.proxies = [
            # 格式: {"http": "http://user:pass@ip:port", "https": "http://user:pass@ip:port"}
        ]
        
        # 确保目录存在
        self._ensure_directory_exists(self.data_dir)
        self._ensure_directory_exists(self.output_dir)
        
        # 初始化数据存储
        self.data = {}
    
    def _normalize_path(self, path):
        """标准化路径，确保跨平台一致性"""
        return os.path.abspath(os.path.expanduser(path))
    
    def _ensure_directory_exists(self, dir_path):
        """确保目录存在，如果不存在则创建它"""
        dir_path = self._normalize_path(dir_path)
        
        logger.info(f"检查目录: {dir_path}")
        
        if os.path.exists(dir_path):
            # 路径已存在，检查是否为目录
            if not os.path.isdir(dir_path):
                logger.warning(f"警告: 路径存在但不是目录: {dir_path}")
                
                try:
                    logger.info(f"尝试删除非目录对象: {dir_path}")
                    os.remove(dir_path)
                    logger.info(f"成功删除非目录对象")
                    
                    # 创建目录
                    os.makedirs(dir_path, exist_ok=True)
                    logger.info(f"成功创建目录: {dir_path}")
                except Exception as e:
                    raise OSError(f"无法删除已存在的非目录对象: {dir_path}, 错误: {e}")
            else:
                # 是目录，检查权限
                self._check_directory_permissions(dir_path)
                logger.info(f"目录已存在且可写: {dir_path}")
        else:
            # 路径不存在，创建目录
            try:
                logger.info(f"目录不存在，正在创建: {dir_path}")
                os.makedirs(dir_path, exist_ok=True)
                logger.info(f"成功创建目录: {dir_path}")
                
                # 验证目录是否真的存在
                if not os.path.exists(dir_path):
                    raise OSError(f"目录创建失败，但没有抛出异常: {dir_path}")
                
                # 检查目录权限
                self._check_directory_permissions(dir_path)
                
            except Exception as e:
                logger.error(f"创建目录 {dir_path} 失败: {e}")
                
                # 尝试创建父目录
                parent_dir = os.path.dirname(dir_path)
                if parent_dir and not os.path.exists(parent_dir):
                    logger.info(f"尝试创建父目录: {parent_dir}")
                    try:
                        os.makedirs(parent_dir, exist_ok=True)
                        logger.info(f"成功创建父目录: {parent_dir}")
                        
                        # 再次尝试创建目标目录
                        os.makedirs(dir_path, exist_ok=True)
                        logger.info(f"成功创建目录: {dir_path}")
                    except Exception as e2:
                        logger.error(f"创建父目录 {parent_dir} 失败: {e2}")
                        raise
                else:
                    raise
    
    def _check_directory_permissions(self, dir_path):
        """检查目录是否有写入权限"""
        # 尝试创建一个临时文件测试写入权限
        test_file = os.path.join(dir_path, ".permission_test.tmp")
        try:
            with open(test_file, 'w') as f:
                f.write("test")
            os.remove(test_file)
            logger.info(f"目录写入权限检查通过: {dir_path}")
        except Exception as e:
            # 获取当前用户信息
            user_info = "未知用户"
            try:
                if os.name == 'nt':  # Windows
                    import getpass
                    user_info = getpass.getuser()
                else:  # Linux/macOS
                    import pwd
                    user_info = pwd.getpwuid(os.getuid()).pw_name
            except:
                pass
            
            # 获取目录权限信息
            permissions = "未知权限"
            try:
                if os.name != 'nt':  # 非Windows系统
                    permissions = oct(os.stat(dir_path).st_mode & 0o777)
            except:
                pass
            
            raise OSError(f"""
                目录没有写入权限: {dir_path}
                用户: {user_info}
                目录权限: {permissions}
                
                解决方案:
                1. 检查目录是否被其他程序占用
                2. 修改目录权限 (Windows: 取消只读属性; Linux/macOS: chmod 755)
                3. 以管理员/root权限运行程序
            """)
    
    def _make_request(self, url, retries=3, timeout=10):
        """发送HTTP请求，处理重试和异常"""
        for attempt in range(retries):
            try:
                # 随机延时，避免请求过于频繁
                wait_time = random.uniform(1, 3)
                logger.info(f"等待 {wait_time:.2f} 秒后请求: {url}")
                time.sleep(wait_time)
                
                # 选择请求配置
                request_kwargs = {
                    'headers': self.headers,
                    'timeout': timeout
                }
                
                # 如果启用代理，随机选择一个
                if self.use_proxy and self.proxies:
                    proxy = random.choice(self.proxies)
                    request_kwargs['proxies'] = proxy
                    logger.info(f"使用代理: {proxy.get('https', proxy.get('http'))}")
                
                # 发送请求
                response = requests.get(url, **request_kwargs)
                response.raise_for_status()  # 检查HTTP状态码
                
                # 检查内容是否有效
                if len(response.text) < 100:  # 内容过短可能表示有问题
                    raise ValueError(f"响应内容过短: {len(response.text)} 字节")
                
                return response
            
            except requests.exceptions.HTTPError as e:
                if response.status_code == 403:
                    logger.warning(f"请求被拒绝 (403): {url}")
                    # 403错误可能需要更长的等待时间
                    time.sleep(10 + attempt * 5)
                elif response.status_code == 429:
                    logger.warning(f"请求过多 (429): {url}")
                    time.sleep(30 + attempt * 10)  # 指数退避
                else:
                    logger.error(f"HTTP错误 {response.status_code}: {url}")
                
                if attempt == retries - 1:
                    raise
            
            except Exception as e:
                logger.error(f"请求出错 ({attempt+1}/{retries}): {e}")
                if attempt == retries - 1:
                    raise
    
    def fetch_sse_data(self):
        """获取上海证券交易所数据"""
        logger.info("正在获取上海证券交易所数据...")
        try:
            response = self._make_request(self.sse_url)
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # 实际的网页解析逻辑需要根据上交所网站的实际结构进行调整
            sse_data = {}
            
            # 查找总览表格
            overview_table = None
            for table in soup.find_all('table'):
                if "上市公司数" in table.text:
                    overview_table = table
                    break
            
            if overview_table:
                rows = overview_table.find_all('tr')
                for row in rows[1:]:  # 跳过表头
                    cells = row.find_all('td')
                    if len(cells) >= 3:
                        market = cells[0].text.strip()
                        company_count = cells[1].text.strip()
                        total_cap = cells[2].text.strip()
                        
                        # 处理年份（假设表格按年份排序，最新的在最前面）
                        year = datetime.now().year
                        
                        if market in ["主板", "科创板"]:
                            if year not in sse_data:
                                sse_data[year] = {}
                            sse_data[year][market] = int(company_count.replace(',', ''))
            
            self.data["sse"] = sse_data
            logger.info(f"上海证券交易所数据获取完成: {sse_data}")
            return sse_data
            
        except Exception as e:
            logger.error(f"获取上海证券交易所数据时出错: {e}")
            return {}
    
    def fetch_szse_data(self):
        """获取深圳证券交易所数据"""
        logger.info("正在获取深圳证券交易所数据...")
        try:
            response = self._make_request(self.szse_url)
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # 实际的网页解析逻辑需要根据深交所网站的实际结构进行调整
            szse_data = {}
            
            # 查找包含上市公司数量的表格或区块
            # 示例：查找包含"上市公司家数"的元素
            company_count_elements = soup.find_all(lambda tag: "上市公司家数" in tag.text)
            
            if company_count_elements:
                # 假设第一个找到的元素包含我们需要的数据
                parent_div = company_count_elements[0].parent
                count_text = parent_div.find('span').text.strip() if parent_div else "0"
                
                # 获取年份
                year = datetime.now().year
                szse_data[year] = {"总计": int(count_text.replace(',', ''))}
                
                # 尝试提取各板块数据
                # ... 实际代码需要根据网站结构调整
                
            self.data["szse"] = szse_data
            logger.info(f"深圳证券交易所数据获取完成: {szse_data}")
            return szse_data
            
        except Exception as e:
            logger.error(f"获取深圳证券交易所数据时出错: {e}")
            return {}
    
    def fetch_bse_data(self):
        """获取北京证券交易所数据"""
        logger.info("正在获取北京证券交易所数据...")
        try:
            # 先请求主页，获取cookies和其他必要信息
            self._make_request("https://www.bse.cn/")
            
            # 再请求统计数据页面
            response = self._make_request(self.bse_url)
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # 实际的网页解析逻辑需要根据北交所网站的实际结构进行调整
            bse_data = {}
            
            # 查找上市公司数量
            company_count = 0
            
            # 示例：查找包含"上市公司家数"的元素
            count_elements = soup.find_all(lambda tag: "上市公司家数" in tag.text)
            
            if count_elements:
                # 提取数量
                count_text = count_elements[0].parent.text.strip() if count_elements[0].parent else "0"
                count_text = count_text.replace("上市公司家数", "").replace("家", "").strip()
                company_count = int(count_text)
                
                # 获取年份
                year = datetime.now().year
                bse_data[year] = {"总计": company_count}
            
            # 尝试从统计页面获取更多数据
            try:
                response_stat = self._make_request(self.bse_stat_url)
                soup_stat = BeautifulSoup(response_stat.text, 'html.parser')
                
                # 提取统计数据
                # ... 实际代码需要根据网站结构调整
                
            except Exception as e:
                logger.warning(f"获取北交所统计数据页面时出错: {e}")
            
            self.data["bse"] = bse_data
            logger.info(f"北京证券交易所数据获取完成: {bse_data}")
            return bse_data
            
        except Exception as e:
            logger.error(f"获取北京证券交易所数据时出错: {e}")
            return {}
    
    def fetch_industry_data(self):
        """获取行业分布数据"""
        logger.info("正在获取行业分布数据...")
        try:
            industry_data = {
                "sse": {},
                "szse": {},
                "bse": {}
            }
            
            # 获取上交所行业分布
            try:
                sse_industry_url = "http://www.sse.com.cn/market/stockdata/industry/"
                response = self._make_request(sse_industry_url)
                soup = BeautifulSoup(response.text, 'html.parser')
                
                # 解析行业分布数据
                industry_table = soup.find('table', class_='tableA')
                if industry_table:
                    rows = industry_table.find_all('tr')
                    for row in rows[1:]:  # 跳过表头
                        cells = row.find_all('td')
                        if len(cells) >= 3:
                            industry = cells[0].text.strip()
                            company_count = cells[1].text.strip()
                            industry_data["sse"][industry] = int(company_count.replace(',', ''))
            
            except Exception as e:
                logger.error(f"获取上交所行业分布数据时出错: {e}")
                # 使用默认数据
                industry_data["sse"] = {"金融": 100, "制造业": 80, "信息技术": 60, "房地产": 40}
            
            # 获取深交所行业分布
            try:
                szse_industry_url = "http://www.szse.cn/market/overview/industry.html"
                response = self._make_request(szse_industry_url)
                soup = BeautifulSoup(response.text, 'html.parser')
                
                # 解析行业分布数据
                # ... 实际代码需要根据网站结构调整
                
            except Exception as e:
                logger.error(f"获取深交所行业分布数据时出错: {e}")
                # 使用默认数据
                industry_data["szse"] = {"制造业": 120, "信息技术": 90, "医药生物": 70, "金融": 50}
            
            # 获取北交所行业分布
            try:
                bse_industry_url = "https://www.bse.cn/disclosure/listedcompanyindustry.html"
                response = self._make_request(bse_industry_url)
                soup = BeautifulSoup(response.text, 'html.parser')
                
                # 解析行业分布数据
                # ... 实际代码需要根据网站结构调整
                
            except Exception as e:
                logger.error(f"获取北交所行业分布数据时出错: {e}")
                # 使用默认数据
                industry_data["bse"] = {"信息技术": 30, "制造业": 40, "材料": 20, "机械设备": 15}
            
            self.data["industry"] = industry_data
            logger.info("行业分布数据获取完成")
            return industry_data
            
        except Exception as e:
            logger.error(f"获取行业分布数据时出错: {e}")
            return {
                "sse": {"金融": 100, "制造业": 80, "信息技术": 60, "房地产": 40},
                "szse": {"制造业": 120, "信息技术": 90, "医药生物": 70, "金融": 50},
                "bse": {"信息技术": 30, "制造业": 40, "材料": 20, "机械设备": 15}
            }
    
    def save_data(self):
        """保存获取的数据到文件"""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        data_file = os.path.join(self.data_dir, f"stock_data_{timestamp}.json")
        
        # 确保目录存在
        self._ensure_directory_exists(os.path.dirname(data_file))
        
        with open(data_file, 'w', encoding='utf-8') as f:
            json.dump(self.data, f, ensure_ascii=False, indent=4)
        
        logger.info(f"数据已保存到 {data_file}")
    
    def analyze_total_companies(self):
        """分析各年度上市公司总数"""
        logger.info("分析各年度上市公司总数...")
        
        # 整合各交易所数据
        years = []
        total_companies = []
        
        # 获取所有可能的年份
        all_years = set()
        for exchange in ["sse", "szse", "bse"]:
            if exchange in self.data and self.data[exchange]:
                all_years.update(self.data[exchange].keys())
        
        # 按年份统计总数
        for year in sorted(all_years):
            total = 0
            for exchange in ["sse", "szse", "bse"]:
                if exchange in self.data and year in self.data[exchange]:
                    # 累加该交易所该年份的所有板块公司数
                    total += sum(self.data[exchange][year].values())
            
            years.append(year)
            total_companies.append(total)
        
        # 创建数据框
        total_df = pd.DataFrame({
            "年份": years,
            "上市公司总数": total_companies
        })
        
        # 保存结果
        output_file = os.path.join(self.output_dir, "total_companies.csv")
        self._ensure_directory_exists(os.path.dirname(output_file))
        total_df.to_csv(output_file, index=False)
        
        # 可视化
        plt.figure(figsize=(10, 6))
        bars = plt.bar(years, total_companies, color='skyblue')
        plt.title('各年度上市公司总数')
        plt.xlabel('年份')
        plt.ylabel('公司数量')
        plt.xticks(years)
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        
        # 为每个柱子添加数值标签
        for bar in bars:
            height = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2., height + 10,
                    f'{height}', ha='center', va='bottom')
        
        # 保存图表
        chart_file = os.path.join(self.output_dir, "total_companies.png")
        self._ensure_directory_exists(os.path.dirname(chart_file))
        plt.savefig(chart_file, dpi=300, bbox_inches='tight')
        plt.close()
        
        logger.info(f"各年度上市公司总数分析完成，结果已保存到 {output_file} 和 {chart_file}")
        return total_df
    
    def analyze_by_exchange(self):
        """按交易所分析上市公司数量"""
        logger.info("按交易所分析上市公司数量...")
        
        # 整合各交易所数据
        years = []
        sse_counts = []
        szse_counts = []
        bse_counts = []
        
        # 获取所有可能的年份
        all_years = set()
        for exchange in ["sse", "szse", "bse"]:
            if exchange in self.data and self.data[exchange]:
                all_years.update(self.data[exchange].keys())
        
        # 按年份统计各交易所公司数
        for year in sorted(all_years):
            sse_count = sum(self.data["sse"][year].values()) if "sse" in self.data and year in self.data["sse"] else 0
            szse_count = sum(self.data["szse"][year].values()) if "szse" in self.data and year in self.data["szse"] else 0
            bse_count = sum(self.data["bse"][year].values()) if "bse" in self.data and year in self.data["bse"] else 0
            
            years.append(year)
            sse_counts.append(sse_count)
            szse_counts.append(szse_count)
            bse_counts.append(bse_count)
        
        # 创建数据框
        exchange_df = pd.DataFrame({
            "年份": years,
            "上交所": sse_counts,
            "深交所": szse_counts,
            "北交所": bse_counts
        })
        
        # 保存结果
        output_file = os.path.join(self.output_dir, "companies_by_exchange.csv")
        self._ensure_directory_exists(os.path.dirname(output_file))
        exchange_df.to_csv(output_file, index=False)
        
        # 可视化
        plt.figure(figsize=(12, 7))
        x = np.arange(len(years))
        width = 0.25
        
        plt.bar(x - width, sse_counts, width, label='上交所', color='skyblue')
        plt.bar(x, szse_counts, width, label='深交所', color='lightgreen')
        plt.bar(x + width, bse_counts, width, label='北交所', color='salmon')
        
        plt.title('各交易所上市公司数量对比')
        plt.xlabel('年份')
        plt.ylabel('公司数量')
        plt.xticks(x, years)
        plt.legend()
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        
        # 保存图表
        chart_file = os.path.join(self.output_dir, "companies_by_exchange.png")
        self._ensure_directory_exists(os.path.dirname(chart_file))
        plt.savefig(chart_file, dpi=300, bbox_inches='tight')
        plt.close()
        
        logger.info(f"按交易所分析完成，结果已保存到 {output_file} 和 {chart_file}")
        return exchange_df
    
    def analyze_industry_distribution(self):
        """分析行业分布"""
        logger.info("分析行业分布...")
        
        industry_data = self.data.get("industry", {})
        
        for exchange, data in industry_data.items():
            if not data:
                continue
                
            # 创建数据框
            industry_df = pd.DataFrame({
                "行业": list(data.keys()),
                "公司数量": list(data.values())
            })
            
            # 保存结果
            output_file = os.path.join(self.output_dir, f"industry_distribution_{exchange}.csv")
            self._ensure_directory_exists(os.path.dirname(output_file))
            industry_df.to_csv(output_file, index=False)
            
            # 可视化
            plt.figure(figsize=(12, 7))
            bars = plt.bar(data.keys(), data.values(), color='lightblue')
            plt.title(f'{exchange.upper()}上市公司行业分布')
            plt.xlabel('行业')
            plt.ylabel('公司数量')
            plt.xticks(rotation=45, ha='right')
            plt.tight_layout()
            
            # 为每个柱子添加数值标签
            for bar in bars:
                height = bar.get_height()
                plt.text(bar.get_x() + bar.get_width()/2., height + 1,
                        f'{height}', ha='center', va='bottom')
            
            # 保存图表
            chart_file = os.path.join(self.output_dir, f"industry_distribution_{exchange}.png")
            self._ensure_directory_exists(os.path.dirname(chart_file))
            plt.savefig(chart_file, dpi=300, bbox_inches='tight')
            plt.close()
        
        logger.info(f"行业分布分析完成，结果已保存到 {self.output_dir} 目录")
        return industry_data
    
    def run_analysis(self):
        """运行完整的分析流程"""
        logger.info("=" * 50)
        logger.info("开始中国上市公司数量年度统计分析...")
        logger.info(f"数据目录: {self.data_dir}")
        logger.info(f"输出目录: {self.output_dir}")
        logger.info("=" * 50)
        
        # 获取数据
        self.fetch_sse_data()
        self.fetch_szse_data()
        self.fetch_bse_data()
        self.fetch_industry_data()
        
        # 保存原始数据
        self.save_data()
        
        # 数据分析
        total_df = self.analyze_total_companies()
        exchange_df = self.analyze_by_exchange()
        industry_data = self.analyze_industry_distribution()
        
        logger.info("=" * 50)
        logger.info("分析完成！所有结果已保存到 output 目录")
        logger.info("=" * 50)
        
        # 返回主要结果
        return {
            "total_companies": total_df,
            "companies_by_exchange": exchange_df,
            "industry_distribution": industry_data
        }

if __name__ == "__main__":
    # 示例：使用自定义目录和代理
    analyzer = ChinaStockAnalyzer(
        data_dir="data",
        output_dir="output",
        use_proxy=False  # 设置为True并配置proxies属性以使用代理
    )
    results = analyzer.run_analysis()

2025-05-25 21:54:47,583 - __main__ - INFO - 检查目录: d:\ifindata\第一次小组作业\data
2025-05-25 21:54:47,584 - __main__ - INFO - 目录写入权限检查通过: d:\ifindata\第一次小组作业\data
2025-05-25 21:54:47,584 - __main__ - INFO - 目录已存在且可写: d:\ifindata\第一次小组作业\data
2025-05-25 21:54:47,584 - __main__ - INFO - 检查目录: d:\ifindata\第一次小组作业\output
2025-05-25 21:54:47,585 - __main__ - INFO - 目录写入权限检查通过: d:\ifindata\第一次小组作业\output
2025-05-25 21:54:47,585 - __main__ - INFO - 目录已存在且可写: d:\ifindata\第一次小组作业\output
2025-05-25 21:54:47,586 - __main__ - INFO - ==================================================
2025-05-25 21:54:47,586 - __main__ - INFO - 开始中国上市公司数量年度统计分析...
2025-05-25 21:54:47,586 - __main__ - INFO - 数据目录: d:\ifindata\第一次小组作业\data
2025-05-25 21:54:47,587 - __main__ - INFO - 输出目录: d:\ifindata\第一次小组作业\output
2025-05-25 21:54:47,587 - __main__ - INFO - ==================================================
2025-05-25 21:54:47,588 - __main__ - INFO - 正在获取上海证券交易所数据...
2025-05-25 21:54:47,588 - __main__ - INFO - 等待 2.20 秒后请求: